In [ ]:

from pyvis import network as net
from IPython.core.display import display, HTML

In [ ]:
import networkx as nx
G = nx.karate_club_graph()

g4 = net.Network(height='400px', width='50%', notebook=True, heading='Zachary’s Karate Club graph')

g4.from_nx(G)

g4.show_buttons(filter_=['physics'])
g4.show('karate.html')


In [ ]:
g4??

In [ ]:
# Standard imports 

from bokeh.io import output_notebook, show
output_notebook()

In [ ]:
# Plot a complex chart with interactive hover in a few lines of code

from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.sampledata.autompg import autompg_clean as df
from bokeh.transform import factor_cmap

df.cyl = df.cyl.astype(str)
df.yr = df.yr.astype(str)

group = df.groupby(by=['cyl', 'mfr'])
source = ColumnDataSource(group)

p = figure(width=800, height=300, title="Mean MPG by # Cylinders and Manufacturer",
           x_range=group, toolbar_location=None, tools="")

p.xgrid.grid_line_color = None
p.xaxis.axis_label = "Manufacturer grouped by # Cylinders"
p.xaxis.major_label_orientation = 1.2

index_cmap = factor_cmap('cyl_mfr', palette=['#2b83ba', '#abdda4', '#ffffbf', '#fdae61', '#d7191c'], 
                         factors=sorted(df.cyl.unique()), end=1)

p.vbar(x='cyl_mfr', top='mpg_mean', width=1, source=source,
       line_color="white", fill_color=index_cmap, 
       hover_line_color="darkgrey", hover_fill_color=index_cmap)

p.add_tools(HoverTool(tooltips=[("MPG", "@mpg_mean"), ("Cyl, Mfr", "@cyl_mfr")]))

show(p)

In [ ]:
import sys
import matplotlib; matplotlib.use("Qt5Agg")

from PyQt5 import QtWidgets, QtCore
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg, NavigationToolbar2QT
from matplotlib.figure import Figure
from netgraph import EditableGraph


class MplCanvas(FigureCanvasQTAgg):
    def __init__(self, parent=None, width=5, height=4, dpi=100):
        super(MplCanvas, self).__init__(Figure(figsize=(width, height), dpi=dpi))
        self.setParent(parent)
        self.ax = self.figure.add_subplot(111)
        self.graph = EditableGraph([(6, 41), (11, 26), (28, 29), (30, 10), (21, 1), (40, 15), (17, 27), (23, 10), (36, 35), (26, 12), (33, 20), (38, 21), (27, 15), (13, 18), (28, 19), (12, 4), (4, 19), (45, 16), (38, 46), (32, 13), (2, 26), (33, 9), (34, 6), (16, 48), (21, 33), (42, 48), (36, 14), (14, 12), (7, 18), (5, 39), (5, 29), (18, 47), (13, 37), (2, 6), (34, 48), (26, 6), (22, 48), (1, 6), (3, 12), (31, 21), (5, 18), (7, 15), (28, 12), (18, 25), (13, 23), (27, 35), (38, 37), (29, 31), (4, 30), (22, 21)], ax=self.ax)


class MainWindow(QtWidgets.QMainWindow):
    def __init__(self, *args, **kwargs):
        super(MainWindow, self).__init__(*args, **kwargs)

        self.canvas = MplCanvas(self, width=5, height=4, dpi=100)

        # Enable key_press_event events:
        # https://github.com/matplotlib/matplotlib/issues/707/#issuecomment-4181799
        self.canvas.setFocusPolicy(QtCore.Qt.ClickFocus)
        self.canvas.setFocus()

        self.toolbar = NavigationToolbar2QT(self.canvas, self)

        widget = QtWidgets.QWidget()
        self.setCentralWidget(widget)

        layout = QtWidgets.QVBoxLayout(widget)
        layout.addWidget(self.toolbar)
        layout.addWidget(self.canvas)


def main():
    app = QtWidgets.QApplication(sys.argv)
    w = MainWindow()
    w.show()
    app.exec_()



main()	

In [ ]:
import networkx as nx
from pyvis.network import Network
import tkinter as tk
from tkinter import ttk

import re






class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Graph Visualization")

        self.notebook = ttk.Notebook(self)
        self.notebook.pack(expand=True, fill='both')

        self.init_graph()

    def init_graph(self):
        G = nx.karate_club_graph()
        self.nt = Network(height='400px', width='100%', directed=False)

        for node in G.nodes:
            self.nt.add_node(node)

        for edge in G.edges:
            self.nt.add_edge(edge[0], edge[1])

        self.nt.set_options("""
        var options = {
          "nodes": {
            "borderWidth": 2
          },
          "edges": {
            "color": {
              "inherit": true
            },
            "smooth": {
              "type": "continuous"
            }
          },
          "physics": {
            "minVelocity": 0.75
          }
        }
        """)

        self.nt.show("graph.html")

        graph_frame = ttk.Frame(self.notebook)
        self.notebook.add(graph_frame, text="Graph")

        self.webview = tk.Text(graph_frame, wrap="word", width=80, height=20)
        self.webview.pack(expand=True, fill='both')

        with open("graph.html") as file:
            html_code = file.read()

        self.webview.insert('1.0', html_code)

#         self.nt.enable_physics(True)
        self.webview.tag_bind("node", "<Button-1>", self.on_node_click)

        pattern = re.compile(rf'>({node})</span>')
        
        for node in G.nodes:
            match = pattern.search(html_code)
            if match:
                start_index = self.webview.search(match.group(1), "1.0", stopindex=tk.END)
                end_index = self.webview.index(f"{start_index} + {len(str(node))}c")
                self.webview.tag_add("node", start_index, end_index)

    def on_node_click(self, event):
        index = self.webview.index(f"@{event.x},{event.y}")
        node = int(self.webview.get(index))

        node_connections = list(self.nt.get_adj_list()[node])

        connections_frame = ttk.Frame(self.notebook)
        self.notebook.add(connections_frame, text=f"Node {node}")

        connections_label = tk.Label(connections_frame, text=f"Node {node} is connected to nodes: {', '.join(map(str, node_connections))}")
        connections_label.pack()

        self.notebook.select(connections_frame)


app = App()
app.mainloop()


In [ ]:
import networkx as nx
from pyvis.network import Network

def on_node_click(node_id):
    print('CLICKED:', node_id)

G = nx.karate_club_graph()
nt = Network(height='400px', width='100%', directed=False)

for node in G.nodes:
    nt.add_node(node, onclick=f"alert('CLICKED: {node}')")

for edge in G.edges:
    nt.add_edge(edge[0], edge[1])

nt.set_options("""
var options = {
  "nodes": {
    "borderWidth": 2
  },
  "edges": {
    "color": {
      "inherit": true
    },
    "smooth": {
      "type": "continuous"
    }
  },
  "physics": {
    "minVelocity": 0.75
  }
}
""")

nt.write_html("graph.html", notebook=False)
nt.show("graph.html")


In [ ]:
import networkx as nx
from pyvis.network import Network

G = nx.karate_club_graph()
nt = Network(height='400px', width='100%', directed=False)

for node in G.nodes:
    nt.add_node(node)

for edge in G.edges:
    nt.add_edge(edge[0], edge[1])

nt.set_options("""
var options = {
  "nodes": {
    "borderWidth": 2
  },
  "edges": {
    "color": {
      "inherit": true
    },
    "smooth": {
      "type": "continuous"
    }
  },
  "physics": {
    "minVelocity": 0.75
  }
}
""")

html_file = "graph.html"
nt.write_html(html_file, notebook=False)

# Add the JavaScript code to the generated HTML file
with open(html_file, 'r') as file:
    html_code = file.read()

js_code = """
<script>
  function on_node_click(node_id) {
    alert('CLICKED: ' + node_id);
  }
</script>
"""

html_code = html_code.replace("</body>", f"{js_code}\n</body>")

with open(html_file, 'w') as file:
    file.write(html_code)

nt.show(html_file)


In [ ]:
!pip install graphistry

In [ ]:
!pip install xarray

In [ ]:
import holoviews as hv
from holoviews import opts, dim
import networkx as nx
import dask.dataframe as dd

from holoviews.operation.datashader import (
    datashade, dynspread, directly_connect_edges, bundle_graph, stack
)
from holoviews.element.graphs import layout_nodes
from datashader.layout import random_layout
from colorcet import fire

hv.extension('bokeh')

keywords = dict(bgcolor='black', width=800, height=800, xaxis=None, yaxis=None)
opts.defaults(opts.Graph(**keywords), opts.Nodes(**keywords), opts.RGB(**keywords))

In [ ]:
# https://stackoverflow.com/questions/63641971/how-to-handle-node-click-in-pyviz-datashader-network-samples

import pandas as pd

import networkx as nx
import matplotlib.pyplot as plt
import graphistry
from pylab import *
import dask.dataframe as dd

class AnnoteFinder:  # thanks to http://www.scipy.org/Cookbook/Matplotlib/Interactive_Plotting
    """
    callback for matplotlib to visit a node (display an annotation) when points are clicked on.  The
    point which is closest to the click and within xtol and ytol is identified.
    """
    def __init__(self, xdata, ydata, annotes, callback = None, threshold=None, axis=None, xtol=None, ytol=None):
        self.data = list(zip(xdata, ydata, annotes))
        if xtol is None: xtol = ((max(xdata) - min(xdata))/float(len(xdata)))/2
        if ytol is None: ytol = ((max(ydata) - min(ydata))/float(len(ydata)))/2
        self.xtol = xtol
        self.ytol = ytol
        if axis is None: axis = gca()
        self.axis= axis
        self.drawnAnnotations = {}
        self.links = []
        self.callback = callback
        self.threshold = threshold if threshold else 1.0e-3

    def __call__(self, event):
        if event.inaxes:
            clickX = event.xdata
            clickY = event.ydata
            if self.axis is None or self.axis==event.inaxes:
                annotes = []
                smallest_x_dist = float('inf')
                smallest_y_dist = float('inf')
                for x,y,a in self.data:
                    if abs(clickX-x)<=smallest_x_dist and abs(clickY-y)<=smallest_y_dist :
                        dx, dy = x - clickX, y - clickY
                        annotes.append((dx*dx+dy*dy,x,y, a) )
                        smallest_x_dist=abs(clickX-x)
                        smallest_y_dist=abs(clickY-y)
                if annotes:
                    annotes.sort() # to select the nearest node
                    distance, x, y, annote = annotes[0]
                    print(distance)
                    if distance < self.threshold:
                        if self.callback:
                            self.callback(annote)

# https://notebooks.azure.com/seanreed1111/projects/PYVIZ1/html/data/maccdc2012_edges.parq
# df = pd.read_parquet('maccdc2012_edges.parq').head(10)

# df = dd.read_parquet('./data/maccdc2012_full_edges.parq').compute()
# edges_df = edges_df.reset_index(drop=True)

def my_callback(node_id):
    print(f'Clicked {node_id}')

# Build your graph
G = nx.karate_club_graph()
pos = nx.spring_layout(G,k=0.1, iterations=20)  # the layout gives us the nodes position x,y,annotes=[],[],[] for key in pos:
x, y, annotes = [], [], []
for key in pos:
    d = pos[key]
    annotes.append(key)
    x.append(d[0])
    y.append(d[1])

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)

nx.draw(G, pos, font_size=6, node_color='skyblue', edge_color='#BB0000', width=0.5, node_size=200, with_labels=True)


af = AnnoteFinder(x, y, annotes, my_callback)
connect('button_press_event', af)

class ZoomPan:
    def __init__(self):
        self.press = None
        self.cur_xlim = None
        self.cur_ylim = None
        self.x0 = None
        self.y0 = None
        self.x1 = None
        self.y1 = None
        self.xpress = None
        self.ypress = None


    def zoom_factory(self, ax, base_scale = 2.):
        def zoom(event):
            cur_xlim = ax.get_xlim()
            cur_ylim = ax.get_ylim()

            xdata = event.xdata # get event x location
            ydata = event.ydata # get event y location

            if event.button == 'down':
                # deal with zoom in
                scale_factor = 1 / base_scale
            elif event.button == 'up':
                # deal with zoom out
                scale_factor = base_scale
            else:
                # deal with something that should never happen
                scale_factor = 1
                print(event.button)

            new_width = (cur_xlim[1] - cur_xlim[0]) * scale_factor
            new_height = (cur_ylim[1] - cur_ylim[0]) * scale_factor

            relx = (cur_xlim[1] - xdata)/(cur_xlim[1] - cur_xlim[0])
            rely = (cur_ylim[1] - ydata)/(cur_ylim[1] - cur_ylim[0])

            ax.set_xlim([xdata - new_width * (1-relx), xdata + new_width * (relx)])
            ax.set_ylim([ydata - new_height * (1-rely), ydata + new_height * (rely)])
            ax.figure.canvas.draw()

        fig = ax.get_figure() # get the figure of interest
        fig.canvas.mpl_connect('scroll_event', zoom)

        return zoom

    def pan_factory(self, ax):
        def onPress(event):
            if event.inaxes != ax: return
            self.cur_xlim = ax.get_xlim()
            self.cur_ylim = ax.get_ylim()
            self.press = self.x0, self.y0, event.xdata, event.ydata
            self.x0, self.y0, self.xpress, self.ypress = self.press

        def onRelease(event):
            self.press = None
            ax.figure.canvas.draw()

        def onMotion(event):
            if self.press is None: return
            if event.inaxes != ax: return
            dx = event.xdata - self.xpress
            dy = event.ydata - self.ypress
            self.cur_xlim -= dx
            self.cur_ylim -= dy
            ax.set_xlim(self.cur_xlim)
            ax.set_ylim(self.cur_ylim)

            ax.figure.canvas.draw()

        fig = ax.get_figure() # get the figure of interest

        # attach the call back
        fig.canvas.mpl_connect('button_press_event',onPress)
        fig.canvas.mpl_connect('button_release_event',onRelease)
        fig.canvas.mpl_connect('motion_notify_event',onMotion)

        #return the function
        return onMotion

scale = 1.1
zp = ZoomPan()
figZoom = zp.zoom_factory(ax, base_scale = scale)
figPan = zp.pan_factory(ax)

plt.show()

In [ ]:
fig